## AMNE 376: A Study of Richter's Kouroi Through Image Clustering

Author: *Kaiyan Zhang, Yash Mali, prAxIs UBC Team*

### 1. Introduction: How Computers See Visual Art?



### 2. Data Exploration

### 3. Image Embedding Using Dino V2

### 4. Image Clustering and Component Analysis

### 5. Conclusion

### Reference
1. Richter, G. M. A. (1970). *Kouroi: Archaic Greek youths: A study of the development of the Kouros type in Greek sculpture.* Phaidon. Accessed through Internet Archive <https://archive.org/details/kouroiarchaicgre0000rich>.